In [188]:
# imports
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Concatenate, Lambda
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from keras.models import Model
from keras import backend as K
from keras import losses
from keras.layers import Layer
from keras.losses import binary_crossentropy
from keras.optimizers import Adam


In [189]:
# import sys
# !mamba install --yes --prefix {sys.prefix} matplotlib

# To train the VAE in the provided code, you'll follow these steps:

1. Prepare your data: Concatenate the metagenomic sequences, abundances, and trinucleotide frequencies into a single DataFrame.
2. Define the VAE architecture: Use the build_vae function to create the VAE model.
3. Preprocess the data: Standardize your data to have zero mean and unit variance, which helps the model converge faster.
4. Train the VAE: Fit the VAE model to your preprocessed data.

## Prepare the data
- load dfs
- concatenate on contig name

In [190]:
# load 3 mer frequencies
df_3mer = pd.read_csv('ERR3211994_3mer.csv')


# Load the metag k15 frequencies
df_mgabund = pd.read_csv('all_mgsearch.abund.csv')
df_mgabund['read_name'] = df_mgabund['query_filename'].str.replace(r'fasta/', '', regex=True)
df_mgabund['read_name'] = df_mgabund['read_name'].str.replace(r'.fa', '', regex=True)
df_mgabund = df_mgabund[['read_name','average_abund']]

# merge
df = df_mgabund.merge(df_3mer, on='read_name')

# remove any nans
df  = df.dropna()

## Define VAE architecture
Latent dimensions: compressed, lower-dimensional representation of the input data learned by the VAE during training.

- Set number of latent dimensions for VAE (compress data into a 10-dimensional latent space)
- get dimensions of input data (num columns/features)

- build the VAE model using input dimensions and latent dimensions. Returns VAE model and encoder part of VAE
vae, encoder = build_vae(input_dim, latent_dim): This line calls the build_vae function to create the VAE model. The build_vae function takes two arguments: input_dim (dimensionality of the input data) and latent_dim (number of latent dimensions). It returns two objects: vae, which is the entire VAE model, and encoder, which is a submodel representing the encoder part of the VAE.

In [191]:
# Define the number of latent dimensions
latent_dim = 1

 # Dimensionality of the input data (numcols)
input_dim = scaled_data.shape[1] 

# build vae model
vae, encoder = build_vae(input_dim, latent_dim)

## Preprocess data

- Standardizes (z-score normalize) data
- Use sklearn standardscaler
- Used the scaler on input data to transform it into standardized data (by mena and std div)

In [192]:
# Scale data 
sequence_names, scaled_data = preprocess_data(df.values)
preprocessed_df = pd.DataFrame(data=np.concatenate([sequence_names.reshape(-1, 1), scaled_data], axis=1),
                                columns=df.columns)
# check for nans
np.isnan(scaled_data).any()

False

In [193]:
sequence_name_list = df['read_name'].tolist()

In [209]:
learning_rate = 0.7 # Adjust this value as needed
optimizer = Adam(learning_rate=learning_rate)

input_dim = scaled_data.shape[1]
vae, encoder = build_vae(input_dim, latent_dim)
vae.compile(optimizer='adam', loss='binary_crossentropy')  # Use default loss function
vae.fit(scaled_data, scaled_data, epochs=50, batch_size=32, verbose=1)

Epoch 1/50
953/953 [==============================] - 1s 470us/step - loss: -4547.2446
Epoch 2/50
953/953 [==============================] - 0s 464us/step - loss: -124656.7812
Epoch 3/50
953/953 [==============================] - 0s 463us/step - loss: -681287.5625
Epoch 4/50
953/953 [==============================] - 0s 462us/step - loss: -2041267.7500
Epoch 5/50
953/953 [==============================] - 0s 462us/step - loss: -4548010.5000
Epoch 6/50
953/953 [==============================] - 0s 462us/step - loss: -8554868.0000
Epoch 7/50
953/953 [==============================] - 0s 460us/step - loss: -14444168.0000
Epoch 8/50
953/953 [==============================] - 0s 476us/step - loss: -22592402.0000
Epoch 9/50
953/953 [==============================] - 0s 479us/step - loss: -33411234.0000
Epoch 10/50
953/953 [==============================] - 0s 496us/step - loss: -47367816.0000
Epoch 11/50
953/953 [==============================] - 0s 461us/step - loss: -64792668.0000
Epoch 12

In [210]:
# Encode the data to the mean of their latent distributions
latent_representations = encoder.predict(scaled_data)

# Cluster the latent representations
clusters, cluster_medoids = online_medoid_clustering(latent_representations)


953/953 [==============================] - 0s 270us/step


In [211]:
len(clusters)

2

In [201]:
len(cluster_medoids)

2

In [202]:
# Create a dictionary to store the sequence names in each cluster
cluster_sequences = {i: [] for i in range(len(clusters))}

# Iterate over each cluster and find the indices of sequences in that cluster
for cluster_idx, cluster in enumerate(clusters):
    for i, latent_representation in enumerate(latent_representations):
        if latent_representation in cluster:
            # Append the sequence name to the corresponding cluster
            cluster_sequences[cluster_idx].append(sequence_name_list[i])

# Print the sequence names in each cluster
for cluster_idx, sequences in cluster_sequences.items():
    print(f"Cluster {cluster_idx}: {sequences}")

In [194]:
# Define the VAE architecture
def build_vae(input_dim, latent_dim):
    # Encoder
    inputs = Input(shape=(input_dim,))
    h = Dense(32, activation='relu')(inputs)
    z_mean = Dense(latent_dim)(h)
    z_log_var = Dense(latent_dim)(h)

    # Sampling layer
    def sampling(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.0)
        return z_mean + K.exp(z_log_var / 2) * epsilon
    
    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

    # Decoder
    decoder_h = Dense(32, activation='relu')
    decoder_out = Dense(input_dim, activation='sigmoid')
    h_decoded = decoder_h(z)
    outputs = decoder_out(h_decoded)

    vae = Model(inputs, outputs)
    encoder = Model(inputs, z_mean)
    
    return vae, encoder



In [195]:
# Function to preprocess data
def preprocess_data(data):
    # Separate the first column (sequence names)
    sequence_names = data[:, 0]
    # Exclude the first column (sequence names) from scaling
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data[:, 1:])  # Exclude the first column
    return sequence_names, scaled_data

# Function for online iterative medoid clustering
def online_medoid_clustering(latent_representations, threshold=0.5):
    clusters = []
    cluster_medoids = []
    
    for point in latent_representations:
        assigned = False
        for i, medoid in enumerate(cluster_medoids):
            if cosine_similarity([point], [medoid])[0][0] > threshold:
                clusters[i].append(point)
                assigned = True
                break
        
        if not assigned:
            clusters.append([point])
            cluster_medoids.append(point)
    
    return clusters, cluster_medoids


